# Last version of model

In [14]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
import pickle

#raw = pd.read_csv(r'../Data/raw/cars.csv', encoding="UTF8")
raw = pd.read_csv(r'Data/raw/cars.csv')

raw['symboling'] = raw['symboling'].astype('category')
for c in raw.columns:
    if raw[c].dtype == 'object': 
        raw[c] = raw[c].astype('category')

raw.drop(['car_ID'], axis=1, inplace=True)
raw.drop(['CarName'], axis=1, inplace=True)

clean=raw.copy()
clean_cat = clean.select_dtypes(include=['category'])
clean_num = clean.select_dtypes(exclude=['category'])
dummied_cat = pd.get_dummies(data=clean_cat, prefix=["sy", "ft", "as", "do", "ca", "dr", "el", "et", "c", "fs"])

num_feat=clean_num.iloc[:,:-1]
price=clean_num.iloc[:,-1]

scalX = StandardScaler()
scaly = StandardScaler()

scalX.fit(num_feat.values)
scaly.fit(price.values.reshape(-1, 1))

df_features=pd.DataFrame(scalX.transform(num_feat.values))
target=pd.DataFrame(scaly.transform(price.values.reshape(-1, 1)))
data_input = pd.concat([dummied_cat, df_features], axis=1)

X = data_input.values
y = target.values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)
Price_prediction_model = LinearRegression(fit_intercept=True)
Price_prediction_model.fit(X_train, y_train)

with open('Pricing_prediction.pkl', 'wb') as file:
    pickle.dump(Price_prediction_model, file)
    
with open('ScalX.pkl', 'wb') as file:
    pickle.dump(scalX, file)

with open('Scaly.pkl', 'wb') as file:
    pickle.dump(scaly, file)

In [79]:
saisie = [1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1,
          0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0 , 
          94, 157, 65, 55, 2000, 90, 3, 3, 10, 80, 5000, 35, 44]

In [80]:
cat = saisie[:44]
num = saisie[44:]
df_num = pd.Series(num)
df_cat = pd.DataFrame(cat)

numscal = pd.DataFrame(scalX.transform(df_num.values.reshape(1, -1)))
entree = pd.concat([df_cat.T, numscal],axis=1).values

prop = Price_prediction_model.predict(entree)
Our_estimation = scaly.inverse_transform(prop[0]).round(2)

In [81]:
Our_estimation

array([16436.46])